# 🚀 YOLOv11s 소나무 피해목 탐지 - 원클릭 학습 시스템

## 🎯 **4단계 원클릭 실행**

### **⚡ 빠른 시작**
1. **🖥️ GPU 설정**: `런타임` → `런타임 유형 변경` → `T4 GPU` 선택
2. **🚀 원클릭 실행**: `런타임` → `모두 실행` 클릭  
3. **☕ 대기**: 약 1-2시간 후 완성!

### **🔄 자동 실행 단계**
- **1단계**: 환경 설정 및 기존 모델 탐지
- **2단계**: YOLOv11s 소나무 전용 학습
- **3단계**: 결과 분석 및 시각화
- **4단계**: 모델 성능 평가

### **🎯 네이밍 규칙**
- **프로젝트**: `pinetree_yolov11s` (유지)
- **실행명**: `damage_detection_YYYYMMDD_HHMM`
- **예시**: `damage_detection_20250916_1430`

### **? 예상 성능**
- **mAP50**: 93%+ (높은 탐지 정확도)
- **Precision**: 90%+ (오탐지 최소화)  
- **Recall**: 85%+ (실제 피해목 탐지)

---

## 🔧 **1단계: 환경 설정 및 기존 모델 탐지**

> 패키지 설치, 기존 학습된 모델 자동 탐지, GPU 설정을 모두 자동화합니다.

In [ ]:
# 🔧 1단계: 환경 설정 및 기존 모델 탐지
import subprocess
import sys
import os
import glob
from datetime import datetime
from pathlib import Path

def install_package(package):
    """패키지 자동 설치"""
    try:
        __import__(package.split('==')[0])
        print(f"✅ {package} 이미 설치됨")
    except ImportError:
        print(f"📦 {package} 설치 중...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} 설치 완료")

print("🚀 YOLOv11s 소나무 피해목 탐지 - 환경 설정")
print("=" * 50)

# 🗂️ Google Drive 마운트 및 ZIP 데이터 자동 압축 해제 (Colab 환경에서만)
try:
    import google.colab
    IN_COLAB = True
    print("📍 Google Colab 환경 감지")
    
    # Google Drive 마운트
    print("🔗 Google Drive 마운트 중...")
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive 마운트 완료")
    
    # 작업 디렉토리 설정
    drive_path = '/content/drive/MyDrive/pinetree_scan'
    if not os.path.exists(drive_path):
        os.makedirs(drive_path, exist_ok=True)
        print(f"📁 작업 디렉토리 생성: {drive_path}")
    
    # 🔍 ZIP 파일 자동 탐지 및 압축 해제 함수
    def find_and_extract_training_zip():
        """Google Drive에서 training_data ZIP 파일을 찾아 압축 해제"""
        import zipfile
        import shutil
        
        print("\n🔍 Google Drive에서 ZIP 데이터셋 탐지 중...")
        
        # ZIP 파일 검색 경로 (우선순위별)
        zip_search_paths = [
            # 1순위: pinetree_scan/training_data 폴더 내의 모든 ZIP 파일
            '/content/drive/MyDrive/pinetree_scan/training_data/*.zip',
            
            # 2순위: pinetree_scan 폴더 내
            '/content/drive/MyDrive/pinetree_scan/training_data.zip',
            '/content/drive/MyDrive/pinetree_scan/dataset.zip', 
            '/content/drive/MyDrive/pinetree_scan/data.zip',
            
            # 3순위: 직접 업로드
            '/content/drive/MyDrive/training_data.zip',
            '/content/drive/MyDrive/dataset.zip',
            '/content/drive/MyDrive/data.zip',
            '/content/drive/MyDrive/yolo_data.zip',
            '/content/drive/MyDrive/pine_data.zip',
            
            # 4순위: 와일드카드 검색
            '/content/drive/MyDrive/*training*.zip',
            '/content/drive/MyDrive/*dataset*.zip',
            '/content/drive/MyDrive/*data*.zip',
            '/content/drive/MyDrive/pinetree_scan/*training*.zip',
            '/content/drive/MyDrive/pinetree_scan/*dataset*.zip'
        ]
        
        found_zips = []
        
        for search_path in zip_search_paths:
            if '*' in search_path:
                # 와일드카드 검색
                import glob
                matches = glob.glob(search_path)
                for match in matches:
                    if os.path.exists(match) and match.endswith('.zip'):
                        found_zips.append(match)
            else:
                # 직접 경로 확인
                if os.path.exists(search_path):
                    found_zips.append(search_path)
        
        if found_zips:
            # 중복 제거
            found_zips = list(set(found_zips))
            
            print(f"📦 발견된 ZIP 파일들:")
            for i, zip_file in enumerate(found_zips, 1):
                file_size = os.path.getsize(zip_file) / 1024 / 1024  # MB
                print(f"  {i}. {os.path.basename(zip_file)} ({file_size:.1f}MB)")
                print(f"     경로: {zip_file}")
            
            # 가장 큰 파일을 선택 (훈련 데이터가 클 가능성이 높음)
            zip_path = max(found_zips, key=os.path.getsize)
            print(f"\n✅ 선택된 파일: {os.path.basename(zip_path)}")
            print(f"📁 경로: {zip_path}")
            
            # 파일 크기 확인
            zip_size = os.path.getsize(zip_path) / 1024 / 1024  # MB
            print(f"📏 크기: {zip_size:.1f}MB")
            
            # 압축 해제 대상 경로
            extract_path = '/content/drive/MyDrive/pinetree_scan/training_data'
            temp_extract_path = '/content/training_data_temp'
            
            try:
                print(f"\n🔄 ZIP 파일 압축 해제 중...")
                
                # 임시 경로에 압축 해제
                if os.path.exists(temp_extract_path):
                    shutil.rmtree(temp_extract_path)
                
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(temp_extract_path)
                
                print(f"✅ 압축 해제 완료: {temp_extract_path}")
                
                # 압축 해제된 내용 확인 및 정리
                extracted_items = os.listdir(temp_extract_path)
                print(f"📋 압축 해제 내용: {extracted_items}")
                
                # training_data 폴더 구조 정리
                final_path = extract_path
                
                # 기존 training_data가 있으면 삭제
                if os.path.exists(final_path):
                    shutil.rmtree(final_path)
                    print(f"🗑️ 기존 데이터 삭제: {final_path}")
                
                # 압축 해제된 내용이 training_data 폴더인지 확인
                if 'training_data' in extracted_items:
                    # training_data 폴더가 포함된 경우
                    source_path = os.path.join(temp_extract_path, 'training_data')
                    shutil.move(source_path, final_path)
                elif len(extracted_items) == 1 and os.path.isdir(os.path.join(temp_extract_path, extracted_items[0])):
                    # 단일 폴더인 경우
                    source_path = os.path.join(temp_extract_path, extracted_items[0])
                    shutil.move(source_path, final_path)
                else:
                    # 파일들이 직접 압축된 경우
                    shutil.move(temp_extract_path, final_path)
                
                # 임시 폴더 정리
                if os.path.exists(temp_extract_path):
                    shutil.rmtree(temp_extract_path)
                
                # 압축 해제 결과 확인
                if os.path.exists(final_path):
                    contents = os.listdir(final_path)
                    print(f"📂 최종 경로: {final_path}")
                    print(f"📋 내용: {contents}")
                    
                    # 기본 구조 확인
                    images_dir = os.path.join(final_path, 'images')
                    labels_dir = os.path.join(final_path, 'labels')
                    
                    if os.path.exists(images_dir):
                        image_count = len([f for f in os.listdir(images_dir) 
                                         if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif', '.tiff'))])
                        print(f"📸 이미지: {image_count}개")
                    
                    if os.path.exists(labels_dir):
                        label_count = len([f for f in os.listdir(labels_dir) 
                                         if f.endswith('.txt')])
                        print(f"🏷️ 라벨: {label_count}개")
                
                return final_path
                
            except Exception as e:
                print(f"❌ ZIP 압축 해제 실패: {e}")
                return None
        
        else:
            print("❌ ZIP 파일을 찾을 수 없습니다.")
            print("💡 다음 경로에 ZIP 파일을 업로드하세요:")
            print("  📦 /content/drive/MyDrive/pinetree_scan/training_data/your_data.zip")
            print("  📦 /content/drive/MyDrive/pinetree_scan/training_data.zip")
            print("  📦 /content/drive/MyDrive/training_data.zip")
            print("  📦 /content/drive/MyDrive/dataset.zip")
            return None
    
    # ZIP 파일 압축 해제 실행
    extracted_data_path = find_and_extract_training_zip()
    
    # 학습 데이터 경로 확인 (압축 해제 결과 포함)
    data_paths = [
        extracted_data_path,  # 압축 해제된 경로 우선
        '/content/drive/MyDrive/pinetree_scan/training_data',
        '/content/training_data',
        '/content/drive/MyDrive/training_data'
    ]
    
    data_path = None
    for path in data_paths:
        if path and os.path.exists(path):
            # data.yaml이 있거나 images 폴더가 있으면 유효한 경로로 간주
            if (os.path.exists(f"{path}/data.yaml") or 
                os.path.exists(f"{path}/images")):
                data_path = path
                print(f"✅ 학습 데이터 발견: {data_path}")
                break
    
    if not data_path:
        print("⚠️ 학습 데이터를 찾을 수 없습니다.")
        print("📋 필요한 파일 구조:")
        print("  📦 ZIP 파일:")
        print("    📂 training_data.zip")
        print("      📂 images/ (이미지 파일들)")
        print("      📂 labels/ (라벨 파일들)")
        print("      📄 data.yaml (선택사항)")
        print("")
        print("  📁 또는 폴더 구조:")
        print("    📂 training_data/")
        print("      📄 data.yaml")
        print("      📂 images/")
        print("      📂 labels/")
        data_path = '/content/training_data'  # 기본값
    
except ImportError:
    IN_COLAB = False
    print("💻 로컬 환경 감지")
    data_path = './data'  # 로컬 기본 경로

# 📦 필수 패키지 설치
packages = [
    "ultralytics",
    "torch", 
    "torchvision",
    "opencv-python",
    "pillow",
    "matplotlib",
    "numpy",
    "pandas",
    "roboflow"  # 데이터셋 관리용 추가
]

print(f"\n📦 필수 패키지 설치:")
for package in packages:
    install_package(package)

# 📚 라이브러리 임포트
import torch
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

print(f"\n🖥️ 시스템 정보:")
print(f"  Python: {sys.version.split()[0]}")
print(f"  PyTorch: {torch.__version__}")
print(f"  CUDA 사용가능: {'✅' if torch.cuda.is_available() else '❌'}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name()
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"  GPU: {gpu_name}")
    print(f"  GPU 메모리: {gpu_memory:.1f}GB")
    
    # GPU 메모리 정리
    torch.cuda.empty_cache()
    print("  🧹 GPU 메모리 정리 완료")
else:
    print("  ⚠️ GPU를 사용할 수 없습니다. 런타임 설정을 확인하세요!")

# 🏆 기존 학습된 모델 자동 탐지 (강화된 버전)
print(f"\n🔍 기존 학습된 모델 탐지:")

def find_best_model():
    """기존 학습된 모델을 찾는 향상된 함수"""
    # 우선순위별 모델 검색 경로
    search_priorities = [
        # 1순위: 최신 학습 결과 (피해목 탐지 전용)
        {
            'priority': 1,
            'description': '최신 피해목 탐지 모델',
            'patterns': [
                'pinetree_yolov11s/damage_detection_*/weights/best.pt',
                '/content/pinetree_yolov11s/damage_detection_*/weights/best.pt',
                '/content/drive/MyDrive/pinetree_scan/results/damage_detection_*/weights/best.pt',
                '/content/drive/MyDrive/best.pt'
            ]
        },
        # 2순위: 일반 학습 결과
        {
            'priority': 2,
            'description': '일반 학습 모델',
            'patterns': [
                'runs/detect/train*/weights/best.pt',
                '/content/runs/detect/train*/weights/best.pt',
                '/content/drive/MyDrive/pinetree_scan/runs/detect/train*/weights/best.pt',
            ]
        },
        # 3순위: 수동 저장 모델
        {
            'priority': 3,
            'description': '수동 저장 모델',
            'patterns': [
                '/content/drive/MyDrive/pinetree_scan/models/best.pt',
                '/content/drive/MyDrive/pinetree_scan/weights/best.pt',
                'models/best.pt',
                'weights/best.pt',
                '/content/best.pt'
            ]
        },
        # 4순위: 기본 모델
        {
            'priority': 4,
            'description': '기본 YOLO 모델',
            'patterns': [
                'yolo11s.pt',
                '/content/yolo11s.pt'
            ]
        }
    ]
    
    found_models = []
    
    for priority_group in search_priorities:
        for pattern in priority_group['patterns']:
            if '*' in pattern:
                matches = glob.glob(pattern)
                for match in matches:
                    if os.path.exists(match):
                        found_models.append({
                            'path': match,
                            'priority': priority_group['priority'],
                            'description': priority_group['description'],
                            'size': os.path.getsize(match) / 1024 / 1024,  # MB
                            'modified': os.path.getctime(match)
                        })
            else:
                if os.path.exists(pattern):
                    found_models.append({
                        'path': pattern,
                        'priority': priority_group['priority'],
                        'description': priority_group['description'],
                        'size': os.path.getsize(pattern) / 1024 / 1024,  # MB
                        'modified': os.path.getctime(pattern)
                    })
    
    if found_models:
        # 우선순위와 수정시간으로 정렬
        found_models.sort(key=lambda x: (x['priority'], -x['modified']))
        return found_models
    
    return None

# 모델 검색 실행
found_models = find_best_model()

if found_models:
    print(f"🎯 발견된 모델: {len(found_models)}개")
    
    # 상위 3개 모델 표시
    for i, model_info in enumerate(found_models[:3]):
        emoji = "🏆" if i == 0 else "🥈" if i == 1 else "🥉"
        print(f"  {emoji} {model_info['description']}")
        print(f"     경로: {model_info['path']}")
        print(f"     크기: {model_info['size']:.1f}MB")
        print(f"     수정: {datetime.fromtimestamp(model_info['modified']).strftime('%Y-%m-%d %H:%M')}")
        
        # 첫 번째 모델 선택
        if i == 0:
            selected_model = model_info['path']
    
    # 선택된 모델 로드 및 검증
    print(f"\n📥 선택된 모델 로드: {os.path.basename(selected_model)}")
    
    try:
        temp_model = YOLO(selected_model)
        
        # 모델 정보 확인
        print(f"✅ 모델 로드 성공!")
        print(f"  📊 모델 정보:")
        
        if hasattr(temp_model, 'model') and hasattr(temp_model.model, 'names'):
            class_names = list(temp_model.model.names.values())
            print(f"    🏷️ 클래스 수: {len(class_names)}")
            print(f"    🏷️ 클래스명: {class_names}")
            
            # 소나무/피해목 관련 클래스 확인
            pine_related = any(keyword in ' '.join(class_names).lower() 
                             for keyword in ['pine', 'damage', 'tree', '피해', '소나무', 'damaged'])
            
            if pine_related:
                print(f"    🌲 소나무 피해목 탐지용 모델 확인!")
            else:
                print(f"    ⚠️ 일반 객체 탐지 모델 (전이학습 예정)")
        
        # 모델 아키텍처 정보
        try:
            model_yaml = temp_model.model.yaml
            if 'backbone' in str(model_yaml):
                print(f"    🏗️ 아키텍처: YOLOv11s")
        except:
            print(f"    🏗️ 아키텍처: YOLO 계열")
            
        existing_model = selected_model
        del temp_model  # 메모리 정리
        
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        print(f"📥 기본 YOLOv11s 모델로 대체")
        existing_model = 'yolo11s.pt'

else:
    print("❌ 기존 학습 모델을 찾을 수 없습니다.")
    print("📥 기본 YOLOv11s 모델 사용")
    existing_model = 'yolo11s.pt'

# 🎯 모델 사용 계획 안내
print(f"\n🎯 모델 사용 계획:")
if 'yolo11s.pt' in existing_model:
    print(f"  📝 기본 YOLOv11s → 소나무 피해목 전이학습")
    print(f"  🎯 예상 학습 시간: 1-2시간 (처음부터)")
    print(f"  📈 예상 성능: 85-95% mAP50")
elif 'damage_detection' in existing_model:
    print(f"  📝 기존 피해목 모델 → 추가 학습/미세조정")
    print(f"  🎯 예상 학습 시간: 30분-1시간 (미세조정)")
    print(f"  📈 예상 성능: 90-98% mAP50")
else:
    print(f"  📝 기존 모델 → 소나무 피해목 전이학습")
    print(f"  🎯 예상 학습 시간: 1시간 (전이학습)")
    print(f"  📈 예상 성능: 87-95% mAP50")

# 🗂️ 출력 디렉토리 설정
if IN_COLAB:
    output_base = '/content/drive/MyDrive/pinetree_scan/results'
else:
    output_base = './results'

os.makedirs(output_base, exist_ok=True)
print(f"\n📁 결과 저장 경로: {output_base}")

# 📊 데이터셋 정보 확인
data_yaml_path = f"{data_path}/data.yaml"
if os.path.exists(data_yaml_path):
    print(f"\n📊 데이터셋 정보:")
    try:
        with open(data_yaml_path, 'r') as f:
            content = f.read()
            print(f"  📄 data.yaml 내용 확인됨")
            if 'names:' in content:
                print(f"  🏷️ 클래스 정보 포함됨")
    except:
        print(f"  ⚠️ data.yaml 읽기 실패")
else:
    print(f"\n⚠️ 데이터셋 파일 없음: {data_yaml_path}")
    print("📋 data.yaml 파일이 필요합니다!")

# 환경 변수 설정
globals()['existing_model'] = existing_model
globals()['data_path'] = data_path
globals()['output_base'] = output_base
globals()['IN_COLAB'] = IN_COLAB
globals()['found_models'] = found_models

print(f"\n✅ 환경 설정 완료!")
print(f"  🏆 사용할 모델: {os.path.basename(existing_model)}")
print(f"  📂 모델 경로: {existing_model}")
print(f"  📊 데이터 경로: {data_path}")
print(f"  📁 결과 경로: {output_base}")
print("=" * 50)

## 🚀 **2단계: YOLOv11s 소나무 전용 학습**

> **FastAPI 전처리 완료된 32px 최적화 데이터로 바로 학습 진행**  
> 기존 모델을 활용한 전이학습으로 소나무 피해목 탐지 성능을 극대화합니다.  
> **네이밍**: `damage_detection_YYYYMMDD_HHMM` 형식으로 자동 생성

In [ ]:
# 🚀 2단계: YOLOv11s 소나무 전용 학습 준비 (T4 GPU 최적화)
print("🌲 YOLOv11s 소나무 피해목 탐지 학습 시작!")
print("🎯 T4 GPU 최적화 + FastAPI 전처리 32px 바운딩 박스")
print("=" * 60)

# ? 메모리 사전 정리 (무한로딩 방지)
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("🧹 GPU 메모리 정리 완료")

# 🔍 1단계 연결 확인 (안전한 기본값)
existing_model = globals().get('existing_model', 'yolo11s.pt')
data_path = globals().get('data_path', '/content/drive/MyDrive/pinetree_scan/training_data')

print(f"📦 사용할 모델: {existing_model}")
print(f"📂 데이터 기본 경로: {data_path}")

# 📊 데이터셋 단계별 탐지 (병목 방지)
print("\n🔍 FastAPI 전처리 데이터셋 탐지 중...")

# T4 GPU 최적화 경로 순서 (빠른 접근부터)
data_search_paths = [
    '/content/drive/MyDrive/pinetree_scan/training_data/data.yaml',
    f"{data_path}/data.yaml",
    '/content/training_data/data.yaml',
    '/content/drive/MyDrive/training_data/data.yaml',
    './data/data.yaml'
]

data_yaml_path = None
for i, path in enumerate(data_search_paths, 1):
    print(f"  {i}. 경로 확인: {path}")
    
    try:
        if os.path.exists(path):
            data_yaml_path = path
            print(f"✅ 데이터셋 발견: {path}")
            break
        else:
            print(f"❌ 없음")
    except Exception as e:
        print(f"❌ 접근 오류: {e}")

if not data_yaml_path:
    print("\n⚠️ data.yaml 파일을 찾을 수 없습니다!")
    print("💡 Google Drive에 FastAPI 전처리 데이터를 업로드했는지 확인하세요.")
    
    # 안전한 기본값 설정
    data_yaml_path = '/content/drive/MyDrive/pinetree_scan/training_data/data.yaml'
    print(f"🔧 기본 경로로 설정: {data_yaml_path}")

# 📊 데이터셋 간단 검증 (병목 방지)
data_valid = False
if data_yaml_path and os.path.exists(data_yaml_path):
    try:
        print(f"\n📊 데이터셋 정보 확인 중...")
        
        # YAML 파일 빠른 검증
        import yaml
        with open(data_yaml_path, 'r', encoding='utf-8') as f:
            data_config = yaml.safe_load(f)
        
        print(f"  ? 데이터 경로: {data_config.get('path', '미설정')}")
        print(f"  🏷️ 클래스: {data_config.get('names', ['피해목'])}")
        
        # 폴더 존재 확인 (개수는 나중에)
        data_dir = os.path.dirname(data_yaml_path)
        train_dir = os.path.join(data_dir, 'train', 'images')
        val_dir = os.path.join(data_dir, 'val', 'images')
        
        if os.path.exists(train_dir) and os.path.exists(val_dir):
            print(f"  ✅ train/val 폴더 확인 완료")
            data_valid = True
        else:
            print(f"  ❌ train/val 폴더 없음")
            
    except Exception as e:
        print(f"  ⚠️ 데이터 검증 실패: {e}")

# 🤖 모델 단계별 로드 (안전한 로딩)
print(f"\n🤖 모델 로드 시작...")
model_ready = False

try:
    print(f"  📥 {existing_model} 로드 중...")
    
    # T4 GPU용 안전한 모델 로드
    model = YOLO(existing_model)
    
    # 모델 검증 (간단히)
    if hasattr(model, 'model'):
        print(f"  ✅ 모델 로드 성공!")
        model_ready = True
    else:
        print(f"  ⚠️ 모델 구조 이상")
        
except Exception as e:
    print(f"  ❌ 모델 로드 실패: {e}")
    print(f"  ? 기본 YOLOv11s 모델로 재시도...")
    
    try:
        model = YOLO('yolo11s.pt')
        existing_model = 'yolo11s.pt'
        model_ready = True
        print(f"  ✅ 기본 모델 로드 성공!")
    except Exception as e2:
        print(f"  ❌ 기본 모델마저 실패: {e2}")

# 🖥️ T4 GPU 최적화 설정
if model_ready:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"\n🖥️ 사용 장치: {device.upper()}")
    
    if device == 'cuda':
        # T4 GPU 메모리 정보
        gpu_props = torch.cuda.get_device_properties(0)
        total_memory = gpu_props.total_memory / 1024**3
        current_memory = torch.cuda.memory_allocated() / 1024**3
        free_memory = total_memory - current_memory
        
        print(f"💾 T4 GPU 메모리: {total_memory:.1f}GB 총 / {free_memory:.1f}GB 사용가능")
        
        # T4 GPU 기본 배치 크기 (안정성 우선)
        batch_size = 8  # T4 GPU + YOLOv11s + 640px 기본값
        
        print(f"🎯 T4 GPU 기본 설정:")
        print(f"  📦 배치 크기: {batch_size} (안정성 우선)")
        print(f"  🎯 이유: 메모리 안전 + 안정적 학습")
        print(f"  💾 메모리 사용량: ~10GB (T4 15GB 기준)")
        
        # 메모리 부족 시 배치 크기 자동 조정
        if free_memory < 10:
            if free_memory >= 8:
                batch_size = 6
                print(f"  ⚠️ 메모리 조정: 배치 크기 6으로 변경")
            elif free_memory >= 6:
                batch_size = 4
                print(f"  ⚠️ 메모리 부족: 배치 크기 4로 안전 모드")
            else:
                batch_size = 2
                print(f"  ❌ 메모리 심각 부족: 배치 크기 2로 최소 모드")
        else:
            print(f"  ✅ 메모리 충분: 배치 크기 {batch_size}로 안정적 학습")
        
        # 추가 메모리 최적화
        torch.backends.cudnn.benchmark = False  # 안정성 우선
        torch.backends.cudnn.deterministic = True
        
    else:
        batch_size = 2  # CPU 모드
        print(f"📦 CPU 모드 배치 크기: {batch_size}")
    
    # 🌐 전역 변수 설정 (다음 단계 연결)
    globals().update({
        'model': model,
        'data_yaml_path': data_yaml_path,
        'batch_size': batch_size,
        'device': device,
        'model_ready': True,
        'data_valid': data_valid,
        'existing_model': existing_model
    })
    
    # ✅ 준비 완료 상태 출력
    print(f"\n✅ 2단계 준비 완료!")
    print(f"  🤖 모델: {existing_model}")
    print(f"  📊 데이터: {'유효' if data_valid else '확인필요'}")
    print(f"  🎯 배치 크기: {batch_size} (T4 최적화)")
    print(f"  💾 메모리: {'안전' if device == 'cuda' and free_memory >= 8 else '주의'}")
    
    if not data_valid:
        print(f"\n⚠️ 데이터 확인 필요:")
        print(f"  1. Google Drive에 training_data 폴더 업로드 확인")
        print(f"  2. data.yaml, train/, val/ 폴더 구조 확인")
        print(f"  3. FastAPI 전처리 완료 확인")

else:
    globals()['model_ready'] = False
    print(f"\n❌ 2단계 준비 실패!")
    print(f"💡 해결방법:")
    print(f"  1. 런타임 재시작 후 다시 시도")
    print(f"  2. 1단계부터 순차 실행")

print("=" * 60)
print("🔄 준비 완료! 다음 셀(학습 실행)을 실행하세요.")

In [ ]:
# 🚀 2단계: YOLOv11s 32px 최적화 학습 실행 (T4 GPU 안전 모드)
print("🚀 2단계: YOLOv11s 32px 최적화 학습 시작!")
print("🎯 T4 GPU 안전 최적화 + 개별 나무 탐지를 위한 32px 바운딩박스")
print("=" * 60)

# 🧹 메모리 사전 정리 (무한로딩 방지)
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print("🧹 GPU 메모리 정리 완료")

# 기존 모델 확인 (준비 단계에서 설정됨)
existing_model = globals().get('existing_model', 'yolo11s.pt')
data_yaml_path = globals().get('data_yaml_path', '/content/drive/MyDrive/pinetree_scan/training_data/data.yaml')
device = globals().get('device', 'cuda' if torch.cuda.is_available() else 'cpu')
model_ready = globals().get('model_ready', False)

print(f"📦 사용 모델: {existing_model}")
print(f"📊 데이터 경로: {data_yaml_path}")
print(f"🖥️ 실행 장치: {device}")
print(f"✅ 모델 준비: {'완료' if model_ready else '미완료'}")

# 2단계 준비 상태 확인
if not model_ready:
    print("❌ 2단계 준비가 완료되지 않았습니다!")
    print("💡 이전 셀(2단계 준비)를 먼저 실행하세요.")
    # 강제 종료하지 않고 기본값으로 진행
    model_ready = True

# 📊 데이터 파일 최종 확인 및 경로 수정
data_exists = os.path.exists(data_yaml_path)
print(f"\n📊 데이터 파일 상태: {'✅ 존재' if data_exists else '❌ 없음'}")

if not data_exists:
    print("⚠️ FastAPI 전처리 데이터를 찾을 수 없습니다!")
    
    # 대체 경로 자동 탐지
    alternative_paths = [
        '/content/drive/MyDrive/pinetree_scan/training_data/data.yaml',
        '/content/drive/MyDrive/training_data/data.yaml', 
        '/content/training_data/data.yaml',
        './data/data.yaml'
    ]
    
    print("🔍 대체 경로 탐색 중...")
    for i, alt_path in enumerate(alternative_paths, 1):
        print(f"  {i}. {alt_path}: {'✅' if os.path.exists(alt_path) else '❌'}")
        if os.path.exists(alt_path):
            data_yaml_path = alt_path
            data_exists = True
            print(f"✅ 대체 경로에서 발견: {alt_path}")
            break
    
    if not data_exists:
        print("❌ 학습 데이터를 찾을 수 없습니다!")
        print("💡 해결방법:")
        print("  1. Google Drive에 training_data 폴더 업로드")
        print("  2. 1단계부터 다시 실행")

# 🔧 data.yaml 경로 수정 함수
def fix_data_yaml_paths(yaml_path):
    """data.yaml 파일의 경로를 현재 위치에 맞게 수정"""
    try:
        import yaml
        
        # 원본 파일 읽기
        with open(yaml_path, 'r', encoding='utf-8') as f:
            data_config = yaml.safe_load(f)
        
        print(f"📄 원본 data.yaml 내용:")
        print(f"  path: {data_config.get('path', '없음')}")
        print(f"  train: {data_config.get('train', '없음')}")
        print(f"  val: {data_config.get('val', '없음')}")
        
        # 데이터 폴더 경로 확인
        data_dir = os.path.dirname(yaml_path)
        print(f"  데이터 폴더: {data_dir}")
        
        # 실제 폴더 구조 확인
        train_images = os.path.join(data_dir, 'train', 'images')
        val_images = os.path.join(data_dir, 'val', 'images')
        train_labels = os.path.join(data_dir, 'train', 'labels')
        val_labels = os.path.join(data_dir, 'val', 'labels')
        
        print(f"🔍 폴더 구조 확인:")
        print(f"  train/images: {'✅' if os.path.exists(train_images) else '❌'}")
        print(f"  val/images: {'✅' if os.path.exists(val_images) else '❌'}")
        print(f"  train/labels: {'✅' if os.path.exists(train_labels) else '❌'}")  
        print(f"  val/labels: {'✅' if os.path.exists(val_labels) else '❌'}")
        
        # 경로 수정
        data_config['path'] = data_dir  # 절대 경로로 설정
        data_config['train'] = 'train/images'  # 상대 경로
        data_config['val'] = 'val/images'      # 상대 경로
        
        # 클래스 정보 확인/설정
        if 'names' not in data_config:
            data_config['names'] = ['damaged_tree']  # 기본 클래스명
        
        if 'nc' not in data_config:
            data_config['nc'] = len(data_config['names'])
        
        # 수정된 파일 저장
        fixed_yaml_path = os.path.join(data_dir, 'data_fixed.yaml')
        with open(fixed_yaml_path, 'w', encoding='utf-8') as f:
            yaml.dump(data_config, f, default_flow_style=False, allow_unicode=True)
        
        print(f"✅ 수정된 data.yaml 생성: {fixed_yaml_path}")
        print(f"📄 수정된 내용:")
        print(f"  path: {data_config['path']}")
        print(f"  train: {data_config['train']}")
        print(f"  val: {data_config['val']}")
        print(f"  names: {data_config['names']}")
        print(f"  nc: {data_config['nc']}")
        
        return fixed_yaml_path
        
    except Exception as e:
        print(f"❌ data.yaml 수정 실패: {e}")
        return yaml_path

# 데이터 YAML 경로 수정 실행
if data_exists:
    print(f"\n🔧 data.yaml 경로 수정 중...")
    fixed_data_yaml_path = fix_data_yaml_paths(data_yaml_path)
    data_yaml_path = fixed_data_yaml_path

# 배치 크기 설정 (GPU/CPU에 따라)
if device == 'cuda':
    # T4 GPU 기본 배치 크기
    batch_size = 8  # T4 GPU + YOLOv11s + 640px 기본값
    
    print(f"\n🎯 T4 GPU 기본 설정:")
    print(f"  📦 배치 크기: {batch_size} (안정성 우선)")
    print(f"  🎯 이유: 메모리 안전 + 안정적 학습")
    print(f"  💾 메모리 사용량: ~10GB (T4 15GB 기준)")
    
    # 메모리 상태 확인 및 자동 조정
    try:
        gpu_props = torch.cuda.get_device_properties(0)
        total_memory = gpu_props.total_memory / 1024**3
        allocated_memory = torch.cuda.memory_allocated() / 1024**3
        free_memory = total_memory - allocated_memory
        
        print(f"  💾 GPU 메모리: {free_memory:.1f}GB 사용 가능")
        
        if free_memory < 10:
            if free_memory >= 8:
                batch_size = 6
                print(f"  ⚠️ 메모리 조정: 배치 크기 6으로 변경")
            elif free_memory >= 6:
                batch_size = 4
                print(f"  ⚠️ 메모리 부족: 배치 크기 4로 안전 모드")
            else:
                batch_size = 2
                print(f"  ❌ 메모리 심각 부족: 배치 크기 2로 최소 모드")
        else:
            print(f"  ✅ 메모리 충분: 배치 크기 {batch_size}로 안정적 학습")
            
    except Exception as e:
        print(f"  ⚠️ GPU 메모리 확인 실패: {e}")
        print(f"  📦 배치 크기 {batch_size} 유지 (기본값)")
        
else:
    batch_size = 2  # CPU 모드
    print(f"\n📦 CPU 모드 배치 크기: {batch_size}")

# 현재 시간으로 실행명 생성
run_name = f"damage_detection_{datetime.now().strftime('%Y%m%d_%H%M')}"

# 프로젝트 폴더 생성
project_dir = "pinetree_yolov11s"
if not os.path.exists(project_dir):
    os.makedirs(project_dir)
    print(f"📁 프로젝트 폴더 생성: {project_dir}")

# 전역 변수 업데이트
globals().update({
    'run_name': run_name,
    'batch_size': batch_size,
    'data_yaml_path': data_yaml_path
})

# 🚀 학습 실행 (데이터 존재할 때만)
if data_exists and model_ready:
    try:
        print(f"\n🤖 모델 로드 중...")
        
        # 모델 로드 (안전 모드)
        if 'model' in globals():
            print("✅ 기존 모델 재사용")
        else:
            model = YOLO(existing_model)
            print(f"✅ 새 모델 로드: {existing_model}")
        
        # T4 GPU 안전 학습 설정 (YOLO 기본값)
        training_config = {
            # 🎯 기본 YOLO 설정
            'data': data_yaml_path,
            'epochs': 200,                  # 기본 에포크
            'patience': 75,                 # 기본 참을성
            'batch': batch_size,
            'imgsz': 640,
            'save': True,
            'save_period': 10,
            'cache': False,                 # T4 안전성 유지
            'device': device,
            'workers': 2,
            'project': project_dir,
            'name': run_name,
            'exist_ok': True,
            'pretrained': True,
            'verbose': True,
            
            # 🧠 기본 YOLO 옵티마이저 설정
            'optimizer': 'SGD',             # 기본 SGD 옵티마이저
            'lr0': 0.001,                   # 기본 학습률
            'lrf': 0.1,                     # 기본 최종 학습률
            'momentum': 0.937,              # YOLO 기본값
            'weight_decay': 0.0005,         # 기본 정규화
            'warmup_epochs': 3,             # 기본 워밍업
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,          # 기본 편향 학습률
            
            # 🎨 기본 데이터 증강
            'hsv_h': 0.015,                 # 기본 색상 변화
            'hsv_s': 0.7,                   # 기본 채도 변화
            'hsv_v': 0.4,                   # 기본 밝기 변화
            'degrees': 0.0,                 # 회전 비활성화
            'translate': 0.1,               # 기본 이동
            'scale': 0.5,                   # 기본 스케일
            'shear': 0.0,                   # 전단 비활성화
            'perspective': 0.0,             # 원근 비활성화
            'flipud': 0.0,                  # 상하 뒤집기 비활성화
            'fliplr': 0.5,                  # 기본 좌우 뒤집기
            'mosaic': 1.0,                  # 기본 모자이크
            'mixup': 0.0,                   # 믹스업 비활성화
            'copy_paste': 0.0,              # 복사-붙여넣기 비활성화
            
            # 🛡️ 안전한 학습 설정
            'amp': False,                   # AMP 비활성화 유지
            'fraction': 1.0,
            'profile': False,
            'freeze': None,
            'dropout': 0.0,
            'val': True,
            'plots': True,
            'save_json': True,
            'save_txt': True,
            'save_conf': True,
            
            # 🌲 기본 YOLO 설정
            'rect': False,
            'single_cls': True,             # 단일 클래스 유지
            'cos_lr': False,                # 코사인 스케줄링 비활성화
            'close_mosaic': 10,             # 기본 모자이크 종료
            'label_smoothing': 0.0,         # 라벨 스무딩 비활성화
            'nbs': 64,                      # 명목 배치 크기
            
            # 🎯 기본 탐지 설정
            'conf': 0.25,                   # 기본 신뢰도
            'iou': 0.45,                    # 기본 IoU
            'max_det': 300,                 # 기본 탐지 수
            
            # 🔬 기본 손실 함수
            'box': 7.5,                     # 기본 박스 손실
            'cls': 0.5,                     # 기본 분류 손실
            'dfl': 1.5,                     # 기본 DFL 손실
        }
        
        # 설정 요약 출력
        print(f"\n🎯 YOLO 기본 설정으로 복원:")
        print(f"  📂 프로젝트: pinetree_yolov11s")
        print(f"  📅 실행명: {run_name}")
        print(f"  🏆 모델: {existing_model}")
        print(f"  📊 데이터: {os.path.basename(data_yaml_path)}")
        print(f"  📏 바운딩박스: 32px (개별 나무 최적화)")
        print(f"  🎯 배치 크기: {batch_size} (T4 안전)")
        print(f"  🧠 옵티마이저: SGD (기본)")
        print(f"  📚 에포크: 200 (기본)")
        print(f"  🎨 증강: 기본 설정 (안정성 우선)")
        print(f"  🔍 탐지: 기본 임계값")
        print(f"  💾 워커: 2개 (병목 방지)")
        print(f"  🔧 경로 수정: data.yaml 절대경로로 수정 완료")
        
        print(f"\n🔥 YOLO 기본 설정 학습 시작!")
        print(f"⏰ 예상 시간: 1-2시간 (기본 학습)")
        print(f"🎯 목표: 안정적 학습, 오탐지 최소화!")
        
        # 🚀 학습 실행
        results = model.train(**training_config)
        
        print(f"\n🎉 YOLO 기본 설정 학습 완료!")
        
        # 결과 분석
        if hasattr(results, 'box') and results.box is not None:
            try:
                map50 = float(results.box.map50)
                map50_95 = float(results.box.map)
                precision = float(results.box.mp)  
                recall = float(results.box.mr)
                
                print(f"📈 최종 성능 지표:")
                print(f"  📈 mAP50: {map50:.4f} ({map50*100:.1f}%)")
                print(f"  📈 mAP50-95: {map50_95:.4f} ({map50_95*100:.1f}%)")
                print(f"  🎯 Precision: {precision:.4f} ({precision*100:.1f}%)")
                print(f"  🔍 Recall: {recall:.4f} ({recall*100:.1f}%)")
                
                # 기본 성능 평가
                print(f"\n🎯 기본 성능 분석:")
                if map50 > 0.7:
                    print("✅ 우수한 성능! 기본 설정으로도 좋은 결과!")
                elif map50 > 0.6:
                    print("✅ 양호한 성능! 실용적으로 사용 가능!")
                elif map50 > 0.5:
                    print("⚠️ 적절한 성능. 추가 학습 고려")
                else:
                    print("❌ 성능 부족. 데이터 확인 필요")
                
                if precision > 0.7:
                    print("🎯 정밀도: 우수 (오탐지 낮음)")
                elif precision > 0.6:
                    print("🎯 정밀도: 양호")
                else:
                    print("🎯 정밀도: 개선 필요")
                
                if recall > 0.7:
                    print("🔍 탐지율: 우수")
                elif recall > 0.6:
                    print("🔍 탐지율: 양호")
                else:
                    print("🔍 탐지율: 개선 필요")
                
                # 성능 데이터 저장
                globals()['model_performance'] = {
                    'mAP50': map50,
                    'mAP50_95': map50_95,
                    'precision': precision,
                    'recall': recall
                }
                
            except Exception as e:
                print(f"⚠️ 성능 지표 분석 실패: {e}")
        
        # 모델 경로 저장
        best_model_path = f"{project_dir}/{run_name}/weights/best.pt"
        globals().update({
            'best_model_path': best_model_path,
            'model': model,
            'training_completed': True
        })
        
        print(f"\n💾 학습 완료 파일:")
        print(f"  🏆 최고 모델: {best_model_path}")
        print(f"  📊 학습 결과: {project_dir}/{run_name}/results.csv")
        print(f"  📈 성능 그래프: {project_dir}/{run_name}/results.png")
        
        print(f"\n✅ T4 GPU 32px 최적화 학습 성공!")
        print(f"🎯 다음: 3단계(검증) → 4단계(백업 및 테스트) 실행!")
        
    except Exception as e:
        print(f"❌ 학습 실행 실패: {e}")
        print(f"💡 해결방법:")
        print(f"  1. 런타임 재시작")
        print(f"  2. GPU 메모리 확인") 
        print(f"  3. 데이터 경로 확인")
        print(f"  4. data.yaml 파일 내용 확인")
        globals()['training_completed'] = False

else:
    print("❌ 학습 실행 조건 미충족!")
    if not data_exists:
        print("  📊 데이터 파일 없음")
    if not model_ready:
        print("  🤖 모델 준비 미완료")
    print("💡 이전 단계들을 완료한 후 다시 시도하세요.")
    globals()['training_completed'] = False

print("=" * 60)

In [ ]:
# 🔍 실시간 mAP 모니터링 도구 (학습 중 변동 분석)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path

def monitor_training_progress():
    """학습 진행상황과 mAP 변동을 실시간으로 모니터링"""
    
    print("📊 실시간 mAP 모니터링 도구")
    print("🎯 학습 중 mAP 변동 원인을 분석합니다")
    print("=" * 50)
    
    # 결과 파일 경로 탐지
    results_paths = []
    
    # 가능한 결과 파일 경로들
    search_patterns = [
        'pinetree_yolov11s/damage_detection_*/results.csv',
        'runs/train/damage_detection_*/results.csv',
        'runs/train/train*/results.csv'
    ]
    
    import glob
    for pattern in search_patterns:
        matches = glob.glob(pattern)
        results_paths.extend(matches)
    
    if not results_paths:
        print("⚠️ 아직 학습 결과 파일이 없습니다")
        print("💡 학습이 시작되면 자동으로 모니터링됩니다")
        return
    
    # 가장 최신 결과 파일 선택
    latest_results = max(results_paths, key=lambda x: Path(x).stat().st_mtime)
    print(f"📄 모니터링 파일: {latest_results}")
    
    try:
        # 결과 데이터 로드
        df = pd.read_csv(latest_results)
        
        if len(df) < 2:
            print("⚠️ 학습 데이터가 부족합니다 (에폭 2개 이상 필요)")
            return
        
        # 주요 지표 추출
        epochs = df['epoch'].values
        
        # mAP 관련 컬럼 찾기
        map_cols = [col for col in df.columns if 'mAP' in col or 'map' in col]
        if not map_cols:
            map_cols = [col for col in df.columns if 'val' in col and 'map' in col.lower()]
        
        if map_cols:
            map_values = df[map_cols[0]].values
            
            print(f"\n📈 mAP 변동 분석 (총 {len(epochs)}개 에폭):")
            print(f"  🎯 현재 mAP: {map_values[-1]:.4f}")
            print(f"  📊 최고 mAP: {np.max(map_values):.4f} (에폭 {epochs[np.argmax(map_values)]})")
            print(f"  📉 최저 mAP: {np.min(map_values):.4f} (에폭 {epochs[np.argmin(map_values)]})")
            print(f"  📊 평균 mAP: {np.mean(map_values):.4f}")
            print(f"  📊 표준편차: {np.std(map_values):.4f}")
            
            # mAP 변동성 분석
            volatility = np.std(map_values) / np.mean(map_values) if np.mean(map_values) > 0 else 0
            
            if volatility > 0.5:
                print(f"  🚨 높은 변동성 ({volatility:.2f}): 학습 불안정")
                print(f"     💡 해결방안: 학습률 감소, 배치 크기 증가")
            elif volatility > 0.3:
                print(f"  ⚠️ 보통 변동성 ({volatility:.2f}): 추가 모니터링 필요")
            else:
                print(f"  ✅ 낮은 변동성 ({volatility:.2f}): 안정적 학습")
            
            # 최근 트렌드 분석 (마지막 10개 에폭)
            if len(map_values) >= 10:
                recent_values = map_values[-10:]
                trend = np.polyfit(range(len(recent_values)), recent_values, 1)[0]
                
                if trend > 0.001:
                    print(f"  📈 상승 트렌드: +{trend:.4f}/에폭")
                elif trend < -0.001:
                    print(f"  📉 하락 트렌드: {trend:.4f}/에폭")
                else:
                    print(f"  ➡️ 평평한 트렌드: 수렴 상태")
        
        # 손실 함수 분석
        loss_cols = [col for col in df.columns if 'loss' in col.lower()]
        if loss_cols:
            train_loss = df[loss_cols[0]].values
            
            print(f"\n📉 손실 함수 분석:")
            print(f"  🎯 현재 손실: {train_loss[-1]:.4f}")
            print(f"  📊 최저 손실: {np.min(train_loss):.4f}")
            
            # 손실과 mAP 상관관계
            if map_cols and len(train_loss) == len(map_values):
                correlation = np.corrcoef(train_loss, map_values)[0, 1]
                print(f"  🔗 손실-mAP 상관관계: {correlation:.3f}")
                
                if correlation < -0.7:
                    print(f"     ✅ 강한 음의 상관관계: 정상적 학습")
                elif correlation < -0.3:
                    print(f"     ⚠️ 약한 상관관계: 과적합 가능성")
                else:
                    print(f"     🚨 비정상적 상관관계: 학습 문제 있음")
        
        # 학습률 분석 (있는 경우)
        lr_cols = [col for col in df.columns if 'lr' in col.lower()]
        if lr_cols:
            lr_values = df[lr_cols[0]].values
            print(f"\n📚 학습률 분석:")
            print(f"  🎯 현재 학습률: {lr_values[-1]:.6f}")
            print(f"  📊 초기 학습률: {lr_values[0]:.6f}")
            
            # 학습률 변화가 급격한지 확인
            lr_changes = np.diff(lr_values)
            max_change = np.max(np.abs(lr_changes))
            
            if max_change > lr_values[0] * 0.1:
                print(f"  ⚠️ 급격한 학습률 변화 감지: 최대 {max_change:.6f}")
                print(f"     💡 이것이 mAP 변동의 원인일 수 있습니다")
        
        print(f"\n💡 mAP 안정화 권장사항:")
        print(f"  1. 배치 크기를 8-16으로 증가")
        print(f"  2. 학습률을 0.005-0.008로 감소")
        print(f"  3. 워밍업 에폭을 5-10으로 증가")
        print(f"  4. 코사인 스케줄링 비활성화")
        print(f"  5. 검증 데이터 품질 점검")
        
    except Exception as e:
        print(f"❌ 결과 파일 분석 실패: {e}")
        print(f"💡 학습이 진행 중이거나 파일 형식 문제일 수 있습니다")

# 모니터링 도구 실행
monitor_training_progress()

print("=" * 50)
print("🔄 학습 중에는 이 셀을 주기적으로 실행하여")
print("📊 mAP 변동을 실시간으로 모니터링하세요!")

## 📊 **3단계: 모델 성능 검증**

> **학습된 YOLOv11s 모델의 성능을 검증하고 결과를 분석합니다**  
> 검증 데이터셋으로 추론을 수행하고 정확도 지표를 확인합니다.

In [ ]:
# 📊 3단계: 모델 성능 검증 실행 (T4 GPU 안전 모드)
print("📊 3단계: 모델 성능 검증 시작! (T4 GPU 최적화)")
print("🎯 학습된 모델의 정확도와 성능을 분석합니다.")
print("⚡ T4 GPU 안전 모드: 메모리 최적화 + 무한로딩 방지")
print("=" * 60)

# T4 GPU 메모리 선제 정리 (무한로딩 방지)
import torch
import gc
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    gc.collect()
    print("🧹 사전 GPU 메모리 정리 완료")

# 학습 완료 상태 확인
training_completed = globals().get('training_completed', False)
training_results = globals().get('training_results', None)
best_model_path = globals().get('best_model_path', None)

print("🔍 학습 상태 확인:")
print(f"  ? 학습 완료: {'✅ 완료' if training_completed else '❌ 미완료'}")
print(f"  📊 학습 결과: {'✅ 있음' if training_results else '❌ 없음'}")
print(f"  🤖 모델 경로: {best_model_path if best_model_path else '❌ 없음'}")

if not training_completed:
    print("\n❌ 학습이 완료되지 않았습니다!")
    
    # 기존 모델 자동 탐지 시도
    print("🔍 기존 학습된 모델 자동 탐지 중...")
    
    import glob
    search_patterns = [
        'pinetree_yolov11s/damage_detection_*/weights/best.pt',
        'runs/train/damage_detection_*/weights/best.pt',
        '/content/pinetree_yolov11s/damage_detection_*/weights/best.pt',
        '/content/drive/MyDrive/pinetree_scan/results/damage_detection_*/weights/best.pt',
    ]
    
    found_models = []
    for pattern in search_patterns:
        matches = glob.glob(pattern)
        found_models.extend(matches)
    
    if found_models:
        # 가장 최신 모델 선택
        best_model_path = max(found_models, key=os.path.getctime)
        print(f"✅ 기존 모델 발견: {best_model_path}")
        
        # 모델 로드
        try:
            model = YOLO(best_model_path)
            training_completed = True
            print("✅ 모델 로드 성공! 검증을 진행합니다.")
        except Exception as e:
            print(f"❌ 모델 로드 실패: {e}")
            training_completed = False
    else:
        print("❌ 학습된 모델을 찾을 수 없습니다!")
        print("💡 2단계(모델 학습)를 먼저 완료하세요.")

if training_completed:
    try:
        # T4 GPU 안전 모델 로드 (타임아웃 방지)
        print("🤖 T4 GPU 안전 모드로 모델 로드 중...")
        
        # 모델이 전역변수에 없으면 로드
        if 'model' not in globals() or best_model_path:
            if best_model_path and os.path.exists(best_model_path):
                # 메모리 정리 후 모델 로드
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                
                model = YOLO(best_model_path)
                print(f"✅ 검증 모델 로드: {best_model_path}")
            elif 'model' in globals():
                print(f"✅ 기존 모델 재사용 (메모리 절약)")
            else:
                print("❌ 사용할 모델이 없습니다!")
                raise Exception("모델 없음")
        else:
            print(f"✅ 기존 모델 재사용")
        
        # 데이터셋 경로 확인 및 안전 모드
        data_yaml_path = globals().get('data_yaml_path', '/content/training_data/data.yaml')
        
        # T4 GPU 안전 검증 모드 활성화
        safe_validation = True
        
        if not os.path.exists(data_yaml_path):
            print(f"⚠️ 데이터셋 파일이 없습니다: {data_yaml_path}")
            
            # 대체 경로 빠른 탐지 (무한로딩 방지)
            quick_paths = [
                '/content/drive/MyDrive/pinetree_scan/training_data/data.yaml',
                '/content/training_data/data.yaml'
            ]
            
            found_data = False
            for quick_path in quick_paths:
                if os.path.exists(quick_path):
                    data_yaml_path = quick_path
                    found_data = True
                    print(f"✅ 대체 경로 발견: {quick_path}")
                    break
            
            if not found_data:
                print("💡 검증 없이 직접 추론 테스트로 진행합니다.")
                safe_validation = False
        
        # 안전 검증 모드 실행
        if safe_validation and os.path.exists(data_yaml_path):
            print(f"\n📊 검증 데이터셋으로 성능 평가 중...")
            print(f"  ? 데이터셋: {data_yaml_path}")
            
            # T4 GPU 안전 메모리 정리
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                torch.cuda.synchronize()
                print("🧹 검증 전 GPU 메모리 정리 완료")
            
            # T4 GPU 최적화 검증 실행 (메모리 안전)
            val_results = model.val(
                data=data_yaml_path,
                imgsz=640,
                batch=2,                      # T4 안전 배치 크기 (8→2)
                conf=0.25,                    # 신뢰도 임계값
                iou=0.6,                     # IoU 임계값
                device=globals().get('device', 'cpu'),
                plots=False,                  # 플롯 비활성화 (메모리 절약)
                save_json=False,             # JSON 비활성화 (속도 향상)
                verbose=False,               # 로그 최소화 (무한로딩 방지)
                workers=1,                   # 워커 최소화
                rect=False,                  # 직사각형 추론 비활성화
                half=False                   # FP16 비활성화 (안정성)
            )
            
            print(f"\n🎉 검증 완료! 성능 지표:")
            
            # 성능 지표 출력
            if hasattr(val_results, 'box'):
                map50 = val_results.box.map50
                map50_95 = val_results.box.map
                precision = val_results.box.mp
                recall = val_results.box.mr
                
                print(f"  📈 mAP50: {map50:.4f} ({map50*100:.1f}%)")
                print(f"  📈 mAP50-95: {map50_95:.4f} ({map50_95*100:.1f}%)")
                print(f"  🎯 Precision: {precision:.4f} ({precision*100:.1f}%)")
                print(f"  ? Recall: {recall:.4f} ({recall*100:.1f}%)")
                
                # 성능 평가
                overall_score = (map50 + precision + recall) / 3
                
                if overall_score > 0.85:
                    performance_grade = "🏆 A+ (탁월)"
                    performance_desc = "상용 수준의 우수한 성능"
                elif overall_score > 0.75:
                    performance_grade = "🌟 A (우수)"
                    performance_desc = "실용적인 수준의 좋은 성능"
                elif overall_score > 0.65:
                    performance_grade = "✅ B (양호)"
                    performance_desc = "적절한 수준의 성능"
                elif overall_score > 0.5:
                    performance_grade = "⚠️ C (보통)"
                    performance_desc = "개선이 필요한 성능"
                else:
                    performance_grade = "❌ D (미흡)"
                    performance_desc = "추가 학습이 필요한 성능"
                
                print(f"\n🎯 종합 성능 평가: {performance_grade}")
                print(f"  📝 평가: {performance_desc}")
                
                # 클래스별 성능 (있는 경우)
                if hasattr(val_results.box, 'ap_class_index') and len(val_results.box.ap_class_index) > 0:
                    print(f"\n? 클래스별 성능:")
                    for i, class_idx in enumerate(val_results.box.ap_class_index):
                        if i < len(val_results.box.ap50):
                            ap50_class = val_results.box.ap50[i]
                            print(f"  🌲 피해목 (클래스 {class_idx}): mAP50 = {ap50_class:.4f}")
                
                # 결과 저장 경로 안내
                if 'run_name' in globals():
                    results_dir = f"pinetree_yolov11s/{run_name}"
                    print(f"\n📁 검증 결과 저장:")
                    print(f"  📂 결과 폴더: {results_dir}")
                    print(f"  🖼️ 혼동행렬: confusion_matrix.png")
                    print(f"  📈 성능곡선: results.png")
                    print(f"  🎯 예측결과: val_batch*.jpg")
                
                # 전역 변수에 검증 결과 저장
                globals()['validation_results'] = val_results
                globals()['validation_completed'] = True
                globals()['model_performance'] = {
                    'mAP50': float(map50),
                    'mAP50_95': float(map50_95),
                    'precision': float(precision),
                    'recall': float(recall),
                    'overall_score': float(overall_score),
                    'grade': performance_grade
                }
                
                validation_completed = True
                
            else:
                print("⚠️ 검증 결과에서 성능 지표를 찾을 수 없습니다.")
                validation_completed = False
        
        else:
            print("⚠️ 안전 검증 모드: 데이터 문제로 검증 건너뜀")
            validation_completed = False
        
        # 검증 후 메모리 정리 (무한로딩 방지)
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
            print("🧹 검증 후 GPU 메모리 정리 완료")
        
        if validation_completed:
            print(f"\n🎯 다음 단계: 4단계(실제 추론) 실행 가능!")
        else:
            print(f"\n⚠️ 검증은 건너뛰었지만 4단계 추론은 가능합니다.")
        
    except Exception as e:
        print(f"❌ 검증 실패: {e}")
        print(f"\n? 문제 해결:")
        print(f"  1. 모델 파일 확인")
        print(f"  2. 데이터셋 경로 확인")
        print(f"  3. GPU 메모리 확인")
        print(f"  4. 런타임 재시작")
        
        globals()['validation_completed'] = False

print("=" * 60)

## 🔍 **4단계: 간단 테스트 및 Google Drive 백업**

> **학습된 모델로 빠른 테스트를 수행하고 Google Drive에 백업합니다**  
> 무한로딩 없는 간단한 테스트와 안전한 백업 시스템입니다.

In [ ]:
# 🎉 4단계: 결과 시각화 + 백업 (디버깅 강화)
print("🎉 4단계: 결과 시각화 + 백업")
print("=" * 30)

import os, shutil, glob
from datetime import datetime

# 딥러닝 결과 폴더 찾기
run_name = globals().get('run_name', '')
folder = None

for pattern in [f"pinetree_yolov11s/{run_name}", "pinetree_yolov11s/damage_detection_*"]:
    matches = glob.glob(pattern)
    if matches:
        folder = matches[0]
        break

if folder and os.path.exists(folder):
    print(f"✅ 결과 폴더: {os.path.basename(folder)}")
    print(f"📂 전체 경로: {folder}")
    
    # ? 폴더 구조 상세 확인
    print(f"\n? 폴더 구조 분석:")
    
    def analyze_folder(path, prefix=""):
        try:
            items = os.listdir(path)
            for item in sorted(items):
                item_path = os.path.join(path, item)
                if os.path.isdir(item_path):
                    print(f"{prefix}📁 {item}/")
                    if prefix == "":  # 1단계 깊이만
                        analyze_folder(item_path, "  ")
                elif item.endswith(('.png', '.jpg', '.jpeg')):
                    size = os.path.getsize(item_path) / 1024  # KB
                    print(f"{prefix}🖼️ {item} ({size:.1f}KB)")
                else:
                    print(f"{prefix}📄 {item}")
        except Exception as e:
            print(f"{prefix}❌ 분석 실패: {e}")
    
    analyze_folder(folder)
    
    # 🖼️ 이미지 파일 직접 검색
    print(f"\n🔍 이미지 파일 직접 검색:")
    
    image_extensions = ['.png', '.jpg', '.jpeg', '.gif']
    found_images = []
    
    for root, dirs, files in os.walk(folder):
        for file in files:
            if any(file.lower().endswith(ext) for ext in image_extensions):
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, folder)
                file_size = os.path.getsize(full_path) / 1024
                found_images.append((file, rel_path, file_size))
                print(f"  🖼️ {rel_path} ({file_size:.1f}KB)")
    
    if not found_images:
        print("  ❌ 이미지 파일이 전혀 없습니다!")
        print("  ? YOLO 훈련이 완전히 끝나지 않았을 수 있습니다")
    
    # 🖼️ 실제 이미지 시각화 시도
    if found_images:
        print(f"\n📊 이미지 시각화 시도:")
        
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg
        from IPython.display import Image, display
        
        displayed_count = 0
        
        for img_name, rel_path, size in found_images[:6]:  # 최대 6개
            img_path = os.path.join(folder, rel_path)
            print(f"\n🖼️ {img_name} 표시 중...")
            
            try:
                # 방법 1: IPython.display.Image
                display(Image(filename=img_path, width=500))
                print(f"  ✅ 성공 (IPython)")
                displayed_count += 1
            except Exception as e1:
                try:
                    # 방법 2: matplotlib
                    fig, ax = plt.subplots(figsize=(10, 6))
                    img = mpimg.imread(img_path)
                    ax.imshow(img)
                    ax.axis('off')
                    ax.set_title(f"{img_name} ({size:.1f}KB)")
                    plt.tight_layout()
                    plt.show()
                    print(f"  ✅ 성공 (matplotlib)")
                    displayed_count += 1
                except Exception as e2:
                    print(f"  ❌ 실패: IPython({str(e1)[:20]}) / matplotlib({str(e2)[:20]})")
        
        print(f"\n✅ {displayed_count}/{len(found_images)} 이미지 표시 완료!")
    
    # 📈 성능 결과
    results_file = os.path.join(folder, 'results.csv')
    if os.path.exists(results_file):
        try:
            import pandas as pd
            df = pd.read_csv(results_file)
            last = df.iloc[-1]
            
            print(f"\n🏆 최종 성능:")
            if 'metrics/mAP50(B)' in df.columns:
                print(f"  📈 mAP50: {last['metrics/mAP50(B)']:.3f}")
            if 'metrics/precision(B)' in df.columns:
                print(f"  🎯 정밀도: {last['metrics/precision(B)']:.3f}")
            if 'metrics/recall(B)' in df.columns:
                print(f"  🔄 재현율: {last['metrics/recall(B)']:.3f}")
        except Exception as e:
            print(f"📊 성능 데이터 로딩 실패: {str(e)[:30]}")
    
    # 💾 Google Drive 백업
    try:
        import google.colab
        
        timestamp = datetime.now().strftime('%m%d_%H%M')
        backup_path = f'/content/drive/MyDrive/pinetree_scan/results/{os.path.basename(folder)}_{timestamp}'
        
        print(f"\n💾 Google Drive 백업 중...")
        
        # 백업 폴더가 이미 있으면 삭제
        if os.path.exists(backup_path):
            shutil.rmtree(backup_path)
            
        shutil.copytree(folder, backup_path)
        
        print(f"✅ 백업 완료!")
        print(f"📁 위치: MyDrive/pinetree_scan/results/{os.path.basename(folder)}_{timestamp}")
        
        globals()['backup_completed'] = True
        
    except ImportError:
        print(f"⚠️ 로컬 환경 - 백업 건너뜀")
    except Exception as e:
        print(f"❌ 백업 실패: {str(e)[:30]}")

else:
    print("❌ 결과 폴더 없음")
    print("💡 2단계를 먼저 실행하세요")

print("=" * 30)